# Weekend Getaway Ranker

## import libray

In [33]:
import pandas as pd
import math

## Load Dataset

In [34]:
data = pd.read_csv("Top Indian Places to Visit.csv")

In [4]:
data.head(2)

,Unnamed: 0,Zone,State,City,Name,Type,Establishment Year,time needed to visit in hrs,Google review rating,Entrance Fee in INR,Airport with 50km Radius,Weekly Off,Significance,DSLR Allowed,Number of google review in lakhs,Best Time to visit
0,0,Northern,Delhi,Delhi,India Gate,War Memorial,1921,0.5,4.6,0,Yes,NaN,Historical,Yes,2.6,Evening
1,1,Northern,Delhi,Delhi,Humayun's Tomb,Tomb,1572,2.0,4.5,30,Yes,NaN,Historical,Yes,0.4,Afternoon


In [5]:
data.describe()

,Unnamed: 0,time needed to visit in hrs,Google review rating,Entrance Fee in INR,Number of google review in lakhs
count,325.000000,325.000000,325.000000,325.000000,325.000000
mean,162.000000,1.807692,4.486154,115.809231,0.408438
std,93.963645,0.971398,0.274580,530.859785,0.646668
min,0.000000,0.500000,1.400000,0.000000,0.010000
25%,81.000000,1.000000,4.400000,0.000000,0.059000
50%,162.000000,1.500000,4.500000,0.000000,0.170000
75%,243.000000,2.000000,4.600000,40.000000,0.500000
max,324.000000,7.000000,4.900000,7500.000000,7.400000


## Calculate null 

In [6]:
print(data.isnull().sum())

Unnamed: 0                            0
Zone                                  0
State                                 0
City                                  0
Name                                  0
Type                                  0
Establishment Year                    0
time needed to visit in hrs           0
Google review rating                  0
Entrance Fee in INR                   0
Airport with 50km Radius              0
Weekly Off                          293
Significance                          0
DSLR Allowed                          0
Number of google review in lakhs      0
Best Time to visit                    0
dtype: int64


In [ ]:
data=data.drop("Weekly Off", axis=1)

Unnamed: 0                          0
Zone                                0
State                               0
City                                0
Name                                0
Type                                0
Establishment Year                  0
time needed to visit in hrs         0
Google review rating                0
Entrance Fee in INR                 0
Airport with 50km Radius            0
Significance                        0
DSLR Allowed                        0
Number of google review in lakhs    0
Best Time to visit                  0
dtype: int64


In [15]:
data.dropna(inplace=True)

## Rename for convenience

In [17]:
data = data.rename(columns={
    "Name": "place",
    "City": "city",
    "Google review rating": "rating",
    "Number of google review in lakhs": "popularity"
})

## Offline City Coordinates

In [18]:
CITY_COORDINATES = {
    "Delhi": (28.6139, 77.2090),
    "Mumbai": (19.0760, 72.8777),
    "Bangalore": (12.9716, 77.5946),
    "Chennai": (13.0827, 80.2707),
    "Kolkata": (22.5726, 88.3639),
    "Hyderabad": (17.3850, 78.4867),
    "Jaipur": (26.9124, 75.7873),
    "Agra": (27.1767, 78.0081),
    "Pune": (18.5204, 73.8567)
}

# Haversine Distance

In [19]:
def haversine(lat1, lon1, lat2, lon2):
    R = 6371
    lat1, lon1, lat2, lon2 = map(math.radians,
                                 [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1

    a = math.sin(dlat/2)**2 + \
        math.cos(lat1) * math.cos(lat2) * \
        math.sin(dlon/2)**2

    return 2 * R * math.asin(math.sqrt(a))

## Normalization

In [20]:
def normalize(col):
    return (col - col.min()) / (col.max() - col.min())


# Recommendation Engine

In [31]:
def recommend_weekend_places(source_city, top_n=3):

    if source_city not in CITY_COORDINATES:
        return "Source city not found."

    src_lat, src_lon = CITY_COORDINATES[source_city]

    # Calculate distance
    data["distance_km"] = data["city"].apply(
        lambda x: haversine(
            src_lat, src_lon,
            CITY_COORDINATES.get(x, (src_lat, src_lon))[0],
            CITY_COORDINATES.get(x, (src_lat, src_lon))[1]
        ) if x in CITY_COORDINATES else None
    ).round(3)

    data.dropna(subset=["distance_km"], inplace=True)

    
    # Normalize factors
    data["rating_norm"] = normalize(data["rating"])
    data["popularity_norm"] = normalize(data["popularity"])
    data["distance_norm"] = 1 - normalize(data["distance_km"])

    # Final score
    data["score"] = (
        data["rating_norm"] * 0.5 +
        data["popularity_norm"] * 0.3 +
        data["distance_norm"] * 0.2
    ).round(2)

    ranked = data.sort_values("score", ascending=False)

    return ranked[[
        "place", "city", "distance_km","rating", "popularity", "score"
    ]].head(top_n)


## User Input

In [32]:
city = input("Enter Source City: ")
print(f"\nTop Weekend Getaways from {city}:\n")
print(recommend_weekend_places(city))

Enter Source City:  Kolkata



Top Weekend Getaways from Kolkata:

                       place     city  distance_km  rating  popularity  score
45  Dakshineswar Kali Temple  Kolkata        0.000     4.7        0.82   0.71
43             Howrah Bridge  Kolkata        0.000     4.6        1.20   0.69
17          Gateway of India   Mumbai     1654.839     4.6        3.60   0.69
